<a href="https://colab.research.google.com/github/spacemaninSeoul/justpractice/blob/main/Selenium_cgv_movie_review_scraping_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CGV 영화 리뷰 스크래핑


## Selenium 및 웹 드라이버 설치

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') # 눈에 보이게 띄우지 않아도 되게끔
chrome_options.add_argument('--disable-dev-shm-usage')

     |████████████████████████████████| 979 kB 21.2 MB/s 
     |████████████████████████████████| 359 kB 45.6 MB/s 
     |████████████████████████████████| 138 kB 57.8 MB/s 
     |████████████████████████████████| 4.0 MB 42.4 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https

In [17]:
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

def get_movie_reviews(url, page_num=10):

  wd = webdriver.Chrome('chromedriver', options=chrome_options)
  wd.get(url)

  writer_list = []
  review_list = []
  date_list = []

  for page_no in range(1, page_num+1):
    try:
      page_ul = wd.find_element_by_id('paging_point')
      page_a = page_ul.find_element_by_link_text(str(page_no)) # 텍스트로 돼있는 형태의 링크를 찾는 것
      page_a.click() # 클릭하게 만드는 메소드 호출
      time.sleep(1)

      writers = wd.find_elements_by_class_name('writer-name')
      writer_list += [writer.text for writer in writers]
      
      reviews = wd.find_elements_by_class_name('box-comment')
      review_list += [review.text for review in reviews]

      dates = wd.find_elements_by_class_name('day')
      date_list += [date.text for date in dates]

      # '다음 10개' 버튼 부분
      if page_no % 10 == 0:
        next_button = page_ul.find_element_by_class_name('btn-paging next')
        next_button.click()
        time.sleep(1)

    except NoSuchElementException:
      break

  movie_review_df = pd.DataFrame({'Writer': writer_list,
                                'Review': review_list,
                                'Date': date_list})
  return movie_review_df

In [18]:
url = 'http://www.cgv.co.kr/movies/detail-view/?midx=85715'
movie_review_df = get_movie_reviews(url, 15)
movie_review_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipyker

,Writer,Review,Date
0,닷컴,두번 보니 감독의 의도가 많이 보여서 더 볼만했었습니다 ㅎㅎ,2022.05.19
1,님보다 여신,와… 너무 너무 별로예요 기대했는데 … 너무 오글? 유치?하달까.. 그리고 너무 질...,2022.05.19
2,HARA♡,기대안했는데 역시 마블~~!,2022.05.19
3,ar**30911,마블을 다 이해하고 보는 편은 아니지만 이번 편은 기승전결이 넘 좋았어요!근데 넘 ...,2022.05.19
4,hy**yu82,기대이상재밌있었어요,2022.05.19
5,tk**6100,엄청 기괴하다 ㅋㅋㅋ 웃김,2022.05.19
6,ah**fk,마블은 진리인데 먼가 스토리나 액션이 따분했네요 여친 저 둘다 졸았습니다ㅠㅠ 아쉽네요,2022.05.19
7,kl**ex,좋아하는 캐릭터 등장만으로도 재밌었지만 뭔가 좀 아쉽기도 하네요,2022.05.19
8,co**s44,닥스내꺼 결혼장려영화입니다,2022.05.19
9,극성맘,4DX 로 봤는데... 얼마나 긴장을 하거 봤는지... 엉덩이... 아플 지경이 였...,2022.05.19


## CGV 상영작 스크래핑

In [21]:
url = 'http://www.cgv.co.kr/movies/'

wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.get(url)

movie_chart = wd.find_element_by_class_name('sect-movie-chart')
contents = movie_chart.find_elements_by_class_name('box-contents')
for content in contents:
  link = content.find_element_by_tag_name('a').get_attribute('href')
  title = content.find_element_by_class_name('title').text
  percent = content.find_element_by_class_name('percent').text
  info = content.find_element_by_class_name('txt-info').text
  print(title, percent, info, link)
  print(get_movie_reviews(link, 2))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_class

범죄도시 2 예매율66.8% 2022.05.18 개봉 http://www.cgv.co.kr/movies/detail-view/?midx=85813


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead


        Writer                                             Review        Date
0          땡근이     아 진짜 속이 후련- 액션 영화 보면서 속 풀린 적 처음입니다 1편보다 더 재밌네요  2022.05.19
1       라돌체비타♥                          역시 마블리 시간가는줄 몰랐음 손석구도 찢었당  2022.05.19
2    es**stlee                                    자유롭게 감상평을 남겨주세요  2022.05.19
3       어나더모제스                                  마동석과 손석구의 연기가 인상적  2022.05.19
4   tj**ns2624                             마동석의 액션영화, 그이상 그이하도 아닌  2022.05.19
5         유아인♥                             짱 재밌어요ㅋㅋㅋ 에스컬레이터씬 기절ㅋㅋ  2022.05.19
6       루이비똥~♡      역시나 믿고보는 마동석님 기대를저버리지 않고 넘 잼나게 봤어욧 연기력도 최고!!!  2022.05.19
7       ce**s1                                   1편보다 재미가 없네요~~ㅠㅠ  2022.05.19
8           씨랑                            재미있는 영화 한편 스트레스를 확 날려보냄  2022.05.19
9     tn**5188                                  너무재밌음....또보러가고싶어요  2022.05.19
10       산내들지니       명불허전. 마블리. 정말 오랜만에 극장에서 시원한 액션 영화 봤네요. 재미있어요  2022.05.19
11     dv**kjl  오! 1편이 너무 강렬했어서 과연 괜찮을까 걱정을 많이했는데 다행이 루즈하지도 않고..

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning:

...

            Writer                                             Review  \
0            ynhow                                      마음을 내려놓으눈 누낌이   
1         영화보느라 한량           마음이 무거워서 보러갔다가 약간 좋아져서 나왔음. 템플스테이 다녀올까보다   
2          jt**601                              명상도하고 생각도 돌아보고 괜찮았습니다   
3     sm**inggipsy                                       Walk with Me   
4          si**ubo                      펀안함을 느끼게 하네요 시간가는 줄 모르고 관람했어요   
5              딜런강                          땡~… 베네딕트 컴버배치의 음성 = 땡~~~~   
6             진한라떼      조급한 현대인에게 느리게 살이가는법을 배우는 기회.. 나레이션이 잘 어울리네요^^   
7   lalalalalalala                                     좋았어요 좋았어요 좋았어요   
8      sh**woo9404                                           감사합니다^♡^   
9             뀽꺙뀨꺙                       근심, 걱정 버리고 현재의 삶을 즐기는 것이 행복♥   
10             사자몫  좋았습니다.보는 내내 마음이 편해 지더군요. 명상앱 쿠폰을 받았는데..잘 이용해서 ...   
11     sh**elberry           불교와 서양의 만남이 신선하면서 신기한 느낌. 잔잔하게 힐링하고 왔어요~   

          Date  
0   2022.05.18  
1   2022.05.18  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning:

           Writer                                             Review  \
0      le**ha0812                                       무난한 영화였습니다..   
1         Neitesy                             일탈이랍시고 이상한 사람과 엮이지 말 것   
2       cr**e5647                                  긴장감이 넘치고 몰입해서 봤어요   
3   Enlightenment                                  어디로 튈지 모르는 인생과 선택   
4        HWANZUNI                               결말이 아쉽지만 재미있는 소재의 영화   
5       캡사이신 아메리카                  모르는 사람이 가자고 하면 가지말라고 부모님 말씀 좀 듣자!   
6            서빈백사                      볼만했어요~~^^ 넘 무섭고 긴장감 있네요.배우연기굿   
7        StylisHY                                     님아 그 선을 넘지 마오.   
8        hannah__                                 햄버거 하나 사주려다가 인생 좇됨   
9             미리내  착하고 순진한 주인공이 낯선 여자에게 베푼 친절이 발목을 잡는 것 같지만, 어느 순...   
10       pe**shot                          달려야할 때 멈추고, 멈춰야할 때 달리는 남자   
11      th**ovier                             어처구니 없는 전개, 어쩌라고 하는 결말   

          Date  
0   2022.05.19  
1   2022.05.19  
2   2022.05.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning:

          Writer                                             Review  \
0        si**aa1                            이야기 구성은 좋은데....뭐가 빠진 느낌   
1           월하가인                                     조금 어설픈 느낌이 크네요   
2        kc**294                                      Yfuhijou전설 내집   
3         tt**82  기대를 많이 해서 그런지 실망한 부분이 좀 .. 한국어로 대사를 하는 부분이 많이 ...   
4     ju**ice21h                                    K 샤머니즘의 전달에 그치는   
5          황금돌고래                                       이딴게 영화인가요 ㄷㄷ   
6            호랑이                                      영화 재밌게 잘 봤습니다   
7     Soul no.22                                         Um...ma...   
8   si**ox926926                                       애매하지만 잘 봤습니다   
9       boss3639                             공포나 호러적인 묘사가 애매모호한 영화!   
10            암실                                     노노 하고싶은 생각이듭니다   
11          지유~♡                                     노노 하고싶은 생각이듭니다   

          Date  
0   2022.05.19  
1   2022.05.19  
2   2022.05.19  
3   2022